In [ ]:
!git clone https://github.com/EmanueleCeglia/PDF-Metadata-Extractor.git

Cloning into 'PDF-Metadata-Extractor'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 49 (delta 19), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (49/49), 470.48 KiB | 17.42 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
%cd PDF-Metadata-Extractor

/content/PDF-Metadata-Extractor


In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt-get install ghostscript -y

## Extract Dates

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from pdf_dates_finder import PDFDatesFinderSemanticSearch  #old method
from pdf_dates_finder import PDFDatesFinderSpace   #new method
import os
import re
from tqdm import tqdm

Download model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B")
model = AutoModelForCausalLM.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B", device_map="auto", load_in_8bit=True)

In [ ]:
def post_process_response_dates(text):
    start_date_pattern = r"Start date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"
    end_date_pattern = r"End date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"

    start_date_match = re.search(start_date_pattern, text)
    end_date_match = re.search(end_date_pattern, text)

    result = {}
    if start_date_match and end_date_match:
        result['Start date'] = start_date_match.group(1)
        result['End date'] = end_date_match.group(1)
    else:
        return None

    return result

Dates extraction with OLD METHOD

In [ ]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extraction = PDFDatesFinderSemanticSearch(full_file_path)
        extraction.load_pdf()
        extraction.process_text()
        dates = extraction.find_dates()

        for value in dates.values():
          for phrase in value.values():
            if len(phrase):
              sentence = phrase[0]

              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=1000)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                extracted_dates[str(file_name)] = response
                print(str(file_name) + ' EXTRACTED!')
              else:
                print(str(file_name) + ' NOT FOUND!')

Dates extraction with NEW METHOD

In [ ]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extractor = PDFDatesFinderSpace(full_file_path)
        pages, tables = extractor.extract_mytext()
        paragraphs = [extractor.identify_paragraphs_space(page) for page in pages]
        #print(paragraphs)

        # keywords filter 1th level
        check_kw = False
        for phrase in paragraphs:
          for senence in phrase:
            if re.search(r'\bperiod\b', senence, re.IGNORECASE):
              check_kw = True

        if check_kw:
          # Use a list comprehension with regex to keep only the phrases that contain the word "period" (case-insensitive)
          paragraphs = [sublist for sublist in paragraphs if any(re.search(r'\bperiod\b', phrase, re.IGNORECASE) for phrase in sublist)]

          # Now, further filter each sublist to keep only the phrases that contain the word "period"
          paragraphs = [[phrase for phrase in sublist if re.search(r'\bperiod\b', phrase, re.IGNORECASE)] for sublist in paragraphs]


        responses = []
        for phrase in paragraphs:
          if len(phrase):
            for sentence in phrase:
              print(sentence)
              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=200)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                  responses.append(response)
                  print(response)

        if responses:
          print(str(file_name) + ' EXTRACTED!')
          extracted_dates[str(file_name)] = responses
        else:
          print(str(file_name) + ' NOT FOUND!')

  0%|          | 0/1 [00:00<?, ?it/s]

policy_20.pdf NOT FOUND!
DDRESS: Avenida Presidente Riesco 5711 Of 1201


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


: With effect from 30th November 2022 12.01 am, local standard time, at the address of the Insured to 31st May 2024 12.01 am, local standard time, at t
{'Start date': '30th November 2022', 'End date': '31st May 2024'}
policy_20.pdf EXTRACTED!
Weapons Exclusion Clause 10th November, 2003 (Cl.370). Marine Cyber Endorsement LMA5403 as attached. Sanction Limitation and Exclusion Clause JH2010/009 as attached. Coronavirus Exclusion LMA 5395, as attached.
itute Fishing Vessel Clauses 20th July, 1987 (Cl.346) eliminating lines 28 to 34 Clause 4 (Termination) – Clause 18 (Liability for Collision) and Clause 20 (Protection and Indemnity). Equipment not attached to the hull, external engine coverage of Full Loss due to the loss of the mother ship only (T.L.V.O.). In respect of Other Vessels: Pursuant to conditions of TLO clauses registered in the registry of insurance policies under POL. 1.92.049, including salvage, salvage expen
policy_20.pdf EXTRACTED!
ls 1st November, 1995 (Cl.280). SECTION I

100%|██████████| 1/1 [05:10<00:00, 310.16s/it]


policy_20.pdf EXTRACTED!


0it [00:00, ?it/s]


In [ ]:
extracted_dates

{'policy_20.pdf': [{'Start date': '30th November 2022',
   'End date': '31st May 2024'}]}